## Scrape GCN circulars

In [21]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
keywords = ["short burst", "short-burst","short-hard", "short/hard", "short hard", "short grb", "short gamma", "short-grb"]

In [22]:
grbs = []
short_flagged = []
for y in range(2004,2020):
    year = str(y)
    main_page = requests.get(f"https://gcn.gsfc.nasa.gov/selected_{year}.html")
    soup = bs(main_page.text, 'html.parser')
    events = soup.find_all('b')
    for event in events:
        if "GRB " in event.text:
            GRB_ID = event.text.split()[1].replace(":","")
            grbs.append(GRB_ID)
            target_url = f"https://gcn.gsfc.nasa.gov/other/{GRB_ID}.gcn3"
            try:
                circulars = requests.get(target_url).text
                if any([search_term in circulars.lower() for search_term in keywords]):
                    short_flagged.append(GRB_ID)
            except:
                print("Couldn't get circulars for GRB",GRB_ID)
        
        else:
            pass
                
    print("Finished "+year+".")

Finished 2004.
Finished 2005.
Finished 2006.
Finished 2007.
Finished 2008.
Finished 2009.
Finished 2010.
Finished 2011.
Finished 2012.
Finished 2013.
Finished 2014.
Finished 2015.
Finished 2016.
Finished 2017.
Finished 2018.
Finished 2019.


In [24]:
new_ones = pd.read_csv("./products/GCN_events_2020-2021.csv",header=None).values.flatten() # events in 2020 and 2021
recent_grbs = []
for entry in new_ones:
    if "GRB " in entry:
        recent_grbs.append(entry)

In [25]:
for event in recent_grbs:
        GRB_ID = event.split()[1].replace(":","")
        grbs.append(GRB_ID)
        target_url = f"https://gcn.gsfc.nasa.gov/other/{GRB_ID}.gcn3"
        try:
            circulars = requests.get(target_url).text
            if any([search_term in circulars.lower() for search_term in keywords]):
                short_flagged.append(GRB_ID)
        except:
            print("Couldn't get circulars for GRB",GRB_ID)

In [5]:
XRT_obs = pd.read_csv("./products/all_XRT_observations.csv")["GRB"].tolist()
already_opt = pd.read_excel("./data/newData.xlsx")["GRB"].dropna().tolist()
current_sample = pd.read_csv("./products/Swift_sGRB_catalog.csv")["GRB"].tolist()
candidates = np.intersect1d(short_flagged, XRT_obs)
# candidates = np.intersect1d(candidates,current_sample)
# ['050202', '050509B', '050709', '050724', '050724A', '050813', '050906', '050925', '051105A', '051114', '051210', '051221A', '060121', '060313', '060502B', '060801', '061006', '061201', '061210', '061217', '070209', '070406', '070429B', '070707', '070714A', '070714B', '070724A', '070729', '070809', '070810B', '070923', '071112B', '071227', '080121', '080123', '080426', '080503', '080702A', '080905A', '080913', '080919', '081024A', '081024B', '081101', '081211B', '081226A', '081226B', '090305', '090305A', '090417A', '090426', '090510', '090515', '090531B', '090607', '090621B', '090715A', '090815C', '090916', '091109B', '091117', '100117A', '100206A', '100213A', '100216A', '100625A', '100628A', '100702A', '100724A', '100816A', '101129A', '101219A', '101224A', '110112A', '110112B', '110402A', '110420B', '111020A', '111117A', '111126A', '120229A', '120305A', '120403A', '120521A', '120630A', '120804A', '120817B', '121226A', '130313A', '130515A', '130603B', '130626A', '130716A', '130822A', '130912A', '131004A', '131125A', '131126A', '140129B', '140320A', '140402A', '140414A', '140516A', '140606A', '140611A', '140619B', '140622A', '140903A', '140930B', '141205A', '141212A', '150101A', '150101B', '150120A', '150301A', '150423A', '150424A', '150710A', '150728A', '150831A', '151127A', '151205B', '151228A', '151229A', '160303A', '160408A', '160410A', '160411A', '160525B', '160601A', '160612A', '160624A', '160709A', '160714A', '160726A', '160821B', '160927A', '161001A', '161104A', '170112A', '170127B', '170325A', '170428A', '170524A', '170728A', '170728B', '171103A', '180204A', '180402A', '180418A', '180715A', '180718A', '180727A', '180805A', '180805B', '181123B', '181126A', '190326A', '190427A', '190610A', '190627A', '191031D', '200219A', '200325A', '200405B', '200411A', '200522A', '200623A', '200716C', '200907B', '201006A', '201214B', '201221D', '210119A', '210323A', '210413B'])

In [17]:
class_table = pd.read_csv("./data/Jespersen_Table1.csv")
class_table["GRB"] = [name[3:] for name in class_table["GRB"]]
jesp_sGRBs = class_table.loc[class_table["Class"]=="S","GRB"]

In [20]:
not_already = np.setdiff1d(candidates,already_opt)
ruled_out = ["050603","050815","051227","060717","061021","070208","080426","100724A","110715A",
             "131002A","140129B","140209A","160228A","191031C","201221A"]
to_do = np.setdiff1d(not_already,ruled_out)
print(len(to_do))
to_do

27


array(['140611A', '160303A', '160410A', '160425A', '161001A', '161129A',
       '171007A', '171211A', '180805B', '200409A', '200512A', '200517A',
       '200729A', '200917A', '201015A', '210217A', '210618A', '210704A',
       '210708A', '210725B', '210726A', '210919A', '211023B', '211106A',
       '211207A', '211221A', '211227A'], dtype='<U7')

In [27]:
for grb in to_do:
    target_url = f"https://gcn.gsfc.nasa.gov/other/{grb}.gcn3"
    try:
        circulars = requests.get(target_url).text
        print(circulars.split("////////////////////////////////////////////////////////////////////////")[1])
    except:
        print("Couldn't get circulars for GRB",grb)
    print("\n")


TITLE:   GCN GRB OBSERVATION REPORT
NUMBER:  2734
SUBJECT: GRB040924 (H3564):  Optical afterglow candidate
DATE:    04/09/24 13:04:49 GMT
FROM:    Derek Fox at CIT  <derekfox@astro.caltech.edu>

D.B. Fox and D.-S. Moon report on behalf of a larger collaboration:  

"We have imaged the localization region of GRB040924 (HETE Trigger
3564) with the robotic Palomar 60-inch telescope and CCD imager in the
R-band.  Comparison of a subset of these images to the digitized sky
survey reveals the presence of a new, bright, stationary source that
shows evidence of variability over the span of our observations.  The
coordinates of the source relative to the GSC-2.2 are:

    RA 02:06:22.45,  Dec +16:06:48.72  (J2000)

where the uncertainty in these coordinates is less than 2 arcsec.  The
source has R>~17 in our images."





TITLE:   GCN GRB OBSERVATION REPORT
NUMBER:  3379
SUBJECT: Swift Detection of the Burst GRB050509
DATE:    05/05/09 02:47:54 GMT
FROM:    Cheryl Pauline Hurkett at Leicester 


TITLE:   GCN GRB OBSERVATION REPORT
NUMBER:  4188
SUBJECT: GRB 051105: Swift detection of a short burst
DATE:    05/11/05 07:17:42 GMT
FROM:    Scott Barthelmy at NASA/GSFC  <Scott@lheamail.gsfc.nasa.gov>

T. Mineo (IASF-Pa/INAF), L. Barbier (GSFC), S. Barthelmy (GSFC),
D. Burrows (PSU), M. Chester (PSU), N. Gehrels (GSFC), J. Kennea (PSU),
H. Krimm (GSFC/USRA), C. Markwardt (GSFC/UMD), D. Palmer (LANL),
G. Tagliaferri (OAB/INAF)
on behalf of the Swift team:

At 06:26:41 UT, Swift-BAT triggered and located GRB 051105 (trigger=162580).
The spacecraft slewed immediately.  The BAT on-board calculated location
is RA,Dec 265.289d,+34.921d {17h 41m 09s,+34d 55' 15"} (J2000),
with an uncertainty of 3 arcmin (radius, 90% containment, stat+sys).
The BAT light curve shows a single spike with a total duration
of less than 128 msec.  The peak count rate was ~3000 counts/sec (15-350 keV),
at 0.0 seconds after the trigger.

XRT began observing at 06:27:49.09 UT, 67.6 sec after BAT Trigger time.
The


TITLE:   GCN GRB OBSERVATION REPORT
NUMBER:  4550
SUBJECT: GRB 060121:  A bright short/hard GRB localized by HETE-2 WXM
DATE:    06/01/21 23:53:39 GMT
FROM:    Don Lamb at U.Chicago  <lamb@oddjob.uchicago.edu>

GRB 060121:  A bright short/hard GRB localized by HETE-2 WXM


M. Arimoto, G. Ricker, J-L. Atteia, N. Kawai, D. Lamb, and S. Woosley,
on behalf of the HETE Science Team;
 
T. Donaghy, E. Fenimore, M. Galassi, C. Graziani, N. Ishikawa,
A. Kobayashi, J. Kotoku, M. Maetou, M. Matsuoka, Y. Nakagawa,
T. Sakamoto, R. Sato, T. Shimokawabe, Y. Shirasaki, S. Sugita,
M. Suzuki, T. Tamagawa, K. Tanaka, and A. Yoshida, on behalf of the
HETE WXM Team;
 
N. Butler, G. Crew, J. Doty, G. Prigozhin, R. Vanderspek,
J. Villasenor, J. G. Jernigan, A. Levine, G. Azzibrouck, J. Braga,
R. Manchanda, G. Pizzichini, and S. Gunasekera, on behalf of the HETE
Operations and HETE Optical-SXC Teams;
 
M. Boer, J-F Olive, J-P Dezalay, and K. Hurley, on behalf of the HETE
FREGATE Team;
 
report:

At 22:24:54.


TITLE:   GCN CIRCULAR
NUMBER:  5699
SUBJECT: GRB 061006: Swift detection of a burst
DATE:    06/10/06 17:13:38 GMT
FROM:    David Palmer at LANL  <palmer@lanl.gov>

P. Schady (MSSL-UCL), D. N. Burrows (PSU), J. R. Cummings (NASA/UMBC),
N. Gehrels (NASA/GSFC), C. Guidorzi (Univ Bicocca&INAF-OAB),
S. T. Holland (GSFC/USRA), S. D. Hunsberger (PSU),
V. Mangano (INAF-IASFPA), D. C. Morris (PSU),
P. T. O'Brien (U Leicester), C. Pagani (PSU),
K. L. Page (U Leicester), D. M. Palmer (LANL),
S. B. Pandey (UCL-MSSL), J. L. Racusin (PSU),
P. Romano (Univ. Bicocca & INAF-OAB), G. Sato (GSFC/ISAS),
M. Stamatikos (NASA/ORAU), R. L. C. Starling (U Leicester),
D. E. Vanden Berk (PSU) and H. Ziaeepour (UCL-MSSL) report on behalf
of the Swift Team:

At 16:45:50 UT, the Swift Burst Alert Telescope (BAT) triggered and
located GRB 061006 (trigger=232585).  Swift slewed immediately to the burst. 
The BAT on-board calculated location is 
RA,Dec 111.090, -79.202 {07h 24m 22s, -79d 12' 05"} (J2000)
with an unc


TITLE:   GCN CIRCULAR
NUMBER:  6074
SUBJECT: GRB 070208: Swift detection of a burst
DATE:    07/02/08 09:34:19 GMT
FROM:    Scott Barthelmy at NASA/GSFC  <scott@lheamail.gsfc.nasa.gov>

G. Sato (GSFC/ISAS), S. D. Barthelmy (GSFC), S. Campana (INAF-OAB),
M. L. Conciatore (ASDC), P. A. Evans (U Leicester),
N. Gehrels (NASA/GSFC), C. Gronwall (PSU),
C. Guidorzi (Univ Bicocca&INAF-OAB), V. Mangano (INAF-IASFPA),
P. T. O'Brien (U Leicester), J. P. Osborne (U Leicester),
K. L. Page (U Leicester), D. M. Palmer (LANL),
P. Romano (Univ. Bicocca & INAF-OAB), G. Stratta (ASDC),
G. Tagliaferri (INAF-OAB), S. D. Vergani (DIAS-DCU) and
H. Ziaeepour (UCL-MSSL) report on behalf of the Swift Team:

At 09:10:34 UT, the Swift Burst Alert Telescope (BAT) triggered and
located GRB 070208 (trigger=259714).  Swift slewed immediately to the burst. 
The BAT on-board calculated location is 
RA, Dec 197.920, +61.959 which is 
   RA(J2000)  =  13h 11m 41s
   Dec(J2000) = +61d 57' 31"
with an uncertainty of 3 arc

...


TITLE:   GCN CIRCULAR
NUMBER:  19594
SUBJECT: IPN Triangulation of GRB 160620A (short/hard)
DATE:    16/06/26 22:29:47 GMT
FROM:    Dmitry Svinkin at Ioffe Institute  <svinkin@mail.ioffe.ru>

K. Hurley, on behalf of the IPN,

I. G. Mitrofanov, D. Golovin, M. L. Litvak, and A. B. Sanin,
on behalf of the HEND-Odyssey GRB team,

S. Golenetskii, R. Aptekar, D. Frederiks, D. Svinkin, A. Kozlova,
and T. Cline on behalf of the Konus-Wind team,

S. Barthelmy, J. Cummings, N. Gehrels, H. Krimm, and D. Palmer,
on behalf of the Swift-BAT team,

A. von Kienlin, X. Zhang, A. Rau, V. Savchenko, E. Bozzo,
and C. Ferrigno, on behalf of the INTEGRAL SPI-ACS GRB team, and

W. Boynton, C. Fellows, K. Harshman, H. Enos, and R. Starr,
on behalf of the GRS-Odyssey GRB team, report:

A short-duration GRB 160620A was detected by Konus-Wind, Swift (BAT), 
INTEGRAL (SPI-ACS), and Mars-Odyssey (HEND) at about 18373 s UT 
(05:06:13). The burst was outside the coded field of view of the BAT.

We have triangulated


TITLE:   GCN CIRCULAR
NUMBER:  20543
SUBJECT: GRB 170127C: Fermi GBM detection
DATE:    17/01/27 16:56:33 GMT
FROM:    Elisabetta Bissaldi at U.Innsbruk/IAPP  <Elisabetta.Bissaldi@uibk.ac.at>

E. Bissaldi (Politecnico & INFN Bari), B. Mailyan and C. Meegan (UAH)
report on behalf of the Fermi GBM Team:


"At 01:35:47.79 UT on 27 January 2017, the Fermi Gamma-Ray Burst Monitor
triggered and located GRB 170127C (trigger 507173752 / 170127067).
The on-ground calculated location, using the GBM trigger data, is

RA, Dec = 336.380, -63.400

(J2000 degrees, equivalent to 22 h 25 m, -63 d 24 '), with an uncertainty
of 1.95 degrees (radius, 1-sigma containment,
statistical only; there is additionally a systematic
error which we have characterized as a core-plus-tail model, with 90% of
GRBs having a 3.7 deg error and a small tail suffering a larger than 10 deg
systematic error. [Connaughton et al. 2015, ApJS, 216, 32]).

The trigger resulted in an Autonomous Repoint Request (ARR)
by the GBM Flig


TITLE:   GCN CIRCULAR
NUMBER:  21257
SUBJECT: IPN Triangulation of GRB 170616A (short/hard)
DATE:    17/06/19 18:56:30 GMT
FROM:    Dmitry Svinkin at Ioffe Institute  <svinkin@mail.ioffe.ru>

K. Hurley, on behalf of the IPN,

I. G. Mitrofanov, D. Golovin, M. L. Litvak, and A. B. Sanin,
on behalf of the HEND-Odyssey GRB team,

D. Svinkin, S. Golenetskii, R. Aptekar, D. Frederiks, A. Kozlova,
and T. Cline on behalf of the Konus-Wind team,

A. von Kienlin, X. Zhang, A. Rau, V. Savchenko, E. Bozzo,
and C. Ferrigno, on behalf of the INTEGRAL SPI-ACS GRB team,

and

W. Boynton, C. Fellows, K. Harshman, H. Enos, and R. Starr,
on behalf of the GRS-Odyssey GRB team, report:

A short-duration, hard-spectrum GRB 170616A was detected by
Konus-Wind, Mars-Odyssey (HEND), and INTEGRAL (SPI-ACS),
at about 57987 s UT (16:06:27).

We have triangulated it to a preliminary, 3 sigma error box whose 
coordinates are:
---------------------------------------------
   RA(2000), deg                 Dec(2000), 


TITLE:   GCN CIRCULAR
NUMBER:  21505
SUBJECT: LIGO/Virgo G298048: Fermi GBM trigger 524666471/170817529: LIGO/Virgo Identification of a possible  gravitational-wave counterpart
DATE:    17/08/17 13:21:42 GMT
FROM:    Reed Clasey Essick at MIT  <ressick@mit.edu>

The LIGO Scientific Collaboration and the Virgo Collaboration report:

The online CBC pipeline (gstlal) has made a preliminary
identification of a GW candidate associated with the time
 of Fermi GBM trigger 524666471/170817529 at gps time 1187008884.47
 (Thu Aug 17 12:41:06 GMT 2017) with RA=186.62deg Dec=-48.84deg and an error radius of 17.45deg.

The candidate is consistent with a neutron star binary coalescence with
False Alarm Rate of ~1/10,000 years. 

An offline analysis is ongoing. Any significant updates will be provided
by a new Circular.

[GCN OPS NOTE(17aug17): Per author's request, the LIGO/VIRGO ID
was added to the beginning of the Subject-line.]





TITLE:   GCN CIRCULAR
NUMBER:  21689
SUBJECT: GRB 170825A: AGIL


TITLE:   GCN CIRCULAR
NUMBER:  23429
SUBJECT: GRB 181121A: AGILE/MCAL detection of a burst
DATE:    18/11/22 08:33:17 GMT
FROM:    Alessandro Ursi at INAF/IAPS  <alessandro.ursi@gmail.com>

*A.Ursi (INAF/IAPS), F. Verrecchia (SSDC, and INAF/OAR), *


*N. Parmiggiani (INAF/IASF-Bo), A. Argan, M. Cardillo, Y. Evangelista, G.
Minervini, G. Piano (INAF/IAPS), M. Tavani (INAF/IAPS, and Univ. Roma Tor
Vergata), F. Lucarelli, C. Pittori, (SSDC, and INAF/OAR), A. Bulgarelli, V.
Fioretti, F. Fuschino (INAF/IASF-Bo), M. Marisaldi (INAF/IASF-Bo, and
Bergen University), M. Pilia, A. Trois (INAF/OA-Cagliari), I. Donnarumma
(ASI), F. Longo (Univ. Trieste and INFN Trieste), A. Giuliani
(INAF/IASF-Mi) report on behalf of the AGILE Team:The AGILE
Mini-CALorimeter (MCAL) detected a short burst at T0 = 2018-11-21
20:43:36.283 +/- 0.01 s (UTC).*
*The light curve shows a sharp peak lasting about 0.16 s, that released a
total number of ~334 counts in the detector (in the 0.4-100 MeV energy
range), above an

In [24]:
classifications = pd.read_csv("./products/new2.csv")
classifications

,GRB,status,optical
0,150906B,real,no
1,150922A,real,no
2,151221A,possible,no
3,160111A,real,no
4,160219A,real,no
...,...,...,...
137,210528A,possible,yes
138,210529A,real,limit
139,210529B,real,yes
140,210601A,real,limit


In [19]:
print(np.unique(classifications[classifications["status"]=="real"]["GRB"].tolist() + ['150424A', '150922A', '151221A', '170219A', '170708A', '170817A', '170825A',
         '180418A', '181121A', '181225A', '190121A', '190606A', '190831B', '050509B',
         '050724', '051103', '051105', '051114', '051211A', '051221', '060427B',
         '060429', '060502B', '070124', '070201', '070429B', '070714B', '070810B',
         '071112B', '080905', '080913', '081211B', '081216', '081223', '090531B',
         '090715A', '090916A', '090927A', '091117A', '091126B', '100213A', '100216A',
         '100816A', '110402A', '110802A', '120811B', '120830A', '140428B', '140604A',
         '150118C', '150906B', '150922A', '160111A', '160219A', '160406A', '160620A',
         '160709A', '160820A', '160822A', '160825A', '170127C', '170206A', '170219A',
         '170222A', '170403A', '170616A', '170708A', '170728B', '170805B', '170816A',
         '170817A', '170825A', '171103A', '180317A', '180529A', '181121A', '181225A',
         '190121A', '190206A', '190606A', '190831B', '200128B', '200219A', '200325A',
         '200327A', '200405B', '200521A', '200623B', '200716C', '200805A', '200815A',
         '200907A', '201214B', '201227A', '210205B', '210307B', '210424B', '210425A',
         '210506A']).tolist())

['050509B', '050724', '051103', '051105', '051114', '051211A', '051221', '060427B', '060429', '060502B', '070124', '070201', '070429B', '070714B', '070810B', '071112B', '080905', '080913', '081211B', '081216', '081223', '090531B', '090715A', '090916A', '090927A', '091117A', '091126B', '100213A', '100216A', '100816A', '110402A', '110802A', '120811B', '120830A', '140428B', '140604A', '150118C', '150424A', '150906B', '150922A', '151221A', '160111A', '160219A', '160406A', '160620A', '160709A', '160820A', '160822A', '160825A', '160829A', '161004A', '170127C', '170206A', '170219A', '170220A', '170222A', '170403A', '170616A', '170708A', '170728B', '170805A', '170805B', '170816A', '170817A', '170822A', '170825A', '170826A', '170827A', '170827B', '170921B', '171030A', '171103A', '171106A', '171223A', '180317A', '180418A', '180529A', '180618A', '180626C', '180715B', '180716A', '180728B', '180824A', '181121A', '181126B', '181222B', '181225A', '190121A', '190206A', '190331C', '190606A', '190626B',

In [20]:
valid = ['050509B', '050724', '051103', '051105', '051114', '051211A', '051221', '060427B', '060429', '060502B', '070124', '070201', '070429B', '070714B', '070810B', '071112B', '080905', '080913', '081211B', '081216', '081223', '090531B', '090715A', '090916A', '090927A', '091117A', '091126B', '100213A', '100216A', '100816A', '110402A', '110802A', '120811B', '120830A', '140428B', '140604A', '150118C', '150424A', '150906B', '150922A', '151221A', '160111A', '160219A', '160406A', '160620A', '160709A', '160820A', '160822A', '160825A', '160829A', '161004A', '170127C', '170206A', '170219A', '170220A', '170222A', '170403A', '170616A', '170708A', '170728B', '170805A', '170805B', '170816A', '170817A', '170822A', '170825A', '170826A', '170827A', '170827B', '170921B', '171030A', '171103A', '171106A', '171223A', '180317A', '180418A', '180529A', '180618A', '180626C', '180715B', '180716A', '180728B', '180824A', '181121A', '181126B', '181222B', '181225A', '190121A', '190206A', '190331C', '190606A', '190626B', '190630A', '190719C', '190724A', '190810A', '190813A', '190830B', '190831B', '190903A', '190913A', '191017C', '191101B', '191116A', '191203A', '191221A', '200103A', '200128A', '200128B', '200129A', '200212A', '200219A', '200221A', '200224C', '200306B', '200307A', '200308A', '200313B', '200325A', '200327A', '200401A', '200405B', '200420A', '200423A', '200501A', '200506B', '200509B', '200514B', '200521A', '200605A', '200623B', '200626A', '200703A', '200706A', '200710A', '200714B', '200716C', '200718A', '200805A', '200815A', '200817A', '200824A', '200826A', '200903C', '200907A', '200908A', '200916B', '200920A', '200920B', '200923A', '200928A', '201103A', '201108A', '201109A', '201111A', '201130A', '201214B', '201221B', '201222A', '201227A', '210124B', '210205B', '210307B', '210326A', '210410A', '210421C', '210424B', '210425A', '210506A', '210510A', '210529A', '210529B', '210601A', '210605A']
valid.sort()